In [ ]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.features import geometry_mask, geometry_window
from rasterio.enums import Resampling
from rasterio.transform import from_bounds
import numpy as np
from rasterio.features import rasterize
from rasterstats import zonal_stats
import os

In [ ]:
lcsf_path = "/Users/rrs/Library/CloudStorage/OneDrive-KTH/KTH/SUPD/0-Degree Project/02-All Codes/ES_Assessment/0-Data/1_predict_newbds_parcels/lcsf.shp"
temp_bp_csv = "/Users/rrs/Library/CloudStorage/OneDrive-KTH/KTH/SUPD/0-Degree Project/02-All Codes/ES_Assessment/0-Data/8-Temperature/BPh_0430.csv"
et0_raster_path = "/Users/rrs/Library/CloudStorage/OneDrive-KTH/KTH/SUPD/0-Degree Project/02-All Codes/ES_Assessment/0-Data/8-Temperature/ET0_SZ.tif"
t_air_ref_raster_path = "/Users/rrs/Library/CloudStorage/OneDrive-KTH/KTH/SUPD/0-Degree Project/02-All Codes/ES_Assessment/0-Data/8-Temperature/TempMon_July2019_SZ.tif"
output_albedo_path = "Users/rrs/Library/CloudStorage/OneDrive-KTH/KTH/SUPD/0-Degree Project/02-All Codes/ES_Assessment/0-Data/8-Temperature/Albedo.tiff"
output_kc_path = "Users/rrs/Library/CloudStorage/OneDrive-KTH/KTH/SUPD/0-Degree Project/02-All Codes/ES_Assessment/0-Data/8-Temperature/Kc.tiff"
output_ETI_path = "Users/rrs/Library/CloudStorage/OneDrive-KTH/KTH/SUPD/0-Degree Project/02-All Codes/ES_Assessment/0-Data/8-Temperature/ETI.tiff"
output_CC_path = "/Users/rrs/Library/CloudStorage/OneDrive-KTH/KTH/SUPD/0-Degree Project/02-All Codes/ES_Assessment/0-Data/8-Temperature/CC.tiff"
output_Temp_path = "/Users/rrs/Library/CloudStorage/OneDrive-KTH/KTH/SUPD/0-Degree Project/02-All Codes/ES_Assessment/0-Data/8-Temperature/Temp.tiff"
output_shapefile_path = "/Users/rrs/Library/CloudStorage/OneDrive-KTH/KTH/SUPD/0-Degree Project/02-All Codes/ES_Assessment/0-Data/8-Temperature/lcsf_result.shp"

In [ ]:
os.makedirs(os.path.dirname(output_albedo_path), exist_ok=True)
os.makedirs(os.path.dirname(output_kc_path), exist_ok=True)
os.makedirs(os.path.dirname(output_ETI_path), exist_ok=True)
os.makedirs(os.path.dirname(output_CC_path), exist_ok=True)
os.makedirs(os.path.dirname(output_Temp_path), exist_ok=True)
os.makedirs(os.path.dirname(output_shapefile_path), exist_ok=True)

In [ ]:
target_crs = "EPSG:2056"

lcsf_gdf = gpd.read_file(lcsf_path)
lcsf_gdf = lcsf_gdf.to_crs(target_crs)

temp_biophysical_df = pd.read_csv(temp_bp_csv, delimiter=";")

In [ ]:
with rasterio.open(et0_raster_path) as et0_src:
    et0_meta = et0_src.meta.copy()
    et0_affine = et0_src.transform
    et0_data = et0_src.read(1)
    et0_bounds = et0_src.bounds
    et0_res = et0_src.res

with rasterio.open(t_air_ref_raster_path) as t_air_ref_src:
    # Resample t_air_ref_data to match et0_data
    t_air_ref_data = t_air_ref_src.read(
        out_shape=(1, et0_data.shape[0], et0_data.shape[1]),
        resampling=Resampling.bilinear
    )[0]
    t_air_ref_affine = et0_src.transform

In [ ]:
lcsf_gdf = lcsf_gdf.merge(temp_biophysical_df, on="Art")

In [ ]:
# 4. Rasterize Albedo and Kc
def rasterize_feature(geom, value, transform, out_shape, all_touched=False):
    return rasterize(
        [(g, v) for g, v in zip(geom, value)],
        out_shape=out_shape,
        transform=transform,
        fill=0,
        all_touched=all_touched,
        dtype=rasterio.float32
    )

albedo_raster = rasterize_feature(lcsf_gdf.geometry, lcsf_gdf['Albedo'], et0_affine, et0_data.shape)
kc_raster = rasterize_feature(lcsf_gdf.geometry, lcsf_gdf['Kc'], et0_affine, et0_data.shape)

# Update the metadata for saving the rasters
et0_meta.update(dtype=rasterio.float32, count=1, compress='lzw')
# Save the Albedo and Kc rasters
with rasterio.open(output_albedo_path, 'w', **et0_meta) as dst:
    dst.write(albedo_raster, 1)

with rasterio.open(output_kc_path, 'w', **et0_meta) as dst:
    dst.write(kc_raster, 1)

In [ ]:
# Calculate ETI
eti_raster = (kc_raster * et0_data) / np.max(et0_data)

In [ ]:
# Save the ETI raster
eti_meta = et0_meta.copy()
eti_meta.update(dtype=rasterio.float32)
with rasterio.open(output_ETI_path, 'w', **eti_meta) as dst:
    dst.write(eti_raster, 1)

In [ ]:
# Calculate Cooling Capacity (CC)
shade_raster = rasterize_feature(lcsf_gdf.geometry, lcsf_gdf['Shade'], et0_affine, et0_data.shape)
cc_raster = 0.6 * shade_raster + 0.2 * albedo_raster + 0.2 * eti_raster

# Save the Cooling Capacity raster
with rasterio.open(output_CC_path, 'w', **eti_meta) as dst:
    dst.write(cc_raster, 1)

In [ ]:
# Calculate Temperature
UHI_max = 2.1 
temp_raster = t_air_ref_data + (1 - cc_raster) * UHI_max

In [ ]:
# Save the Temperature raster
with rasterio.open(output_Temp_path, 'w', **eti_meta) as dst:
    dst.write(temp_raster, 1)

In [ ]:
# Aggregate raster values to polygons

# Add "CC" attribute to shapefile
lcsf_gdf['CC'] = [x['mean'] for x in zonal_stats(lcsf_gdf, cc_raster, affine=et0_affine, stats=['mean'])]

# Add "temperature" attribute to shapefile
lcsf_gdf['temperature'] = [x['mean'] for x in zonal_stats(lcsf_gdf, temp_raster, affine=et0_affine, stats=['mean'])]

In [ ]:
# Save the updated shapefile
lcsf_gdf.to_file(output_shapefile_path)
print(f"Updated shapefile saved to: {output_shapefile_path}")